В этом задании Вам предлагается решить проблему классификации текстов разными методами.

Среди таких методов мы можем предложить Вам:

1) Простой Байесовский классификатор на основе мультиномиальной модели или модели Бернулли

>Достоинства: идейная простота и простота реализации, неплохая интерпретируемость

>Недостатки: относительно слабая предсказательная способность

> Frameworks: `numpy`

2) Логистическая регрессия на основе векторов TF-IDF

>Достоинства: достаточно высокая скорость обучения, простой метод составления эмбеддингов

>Недостатки: также довольно слабая предсказательная способность, слишком высокая размерность задачи

> Frameworks: `sklearn`, `numpy`

3) Логистическая регрессия или нейронная сеть + word2vec embeddings

> Достоинства: оптимальная размерность эмбеддингов, довольно простые модели, сравнительно неплохое качество

> Недостатки: устаревший метод построения эмбеддингов. Эмбеддинги не контекстуальные

> Frameworks: `gensim`, `pytorch`, `sklearn`

4) Рекуррентная нейронная сеть + word2vec:

> Достоинства: Более современная нейронная сеть

> Недостатки: недоступно распараллеливание

> Frameworks: `pytorch`, `gensim`

5) ELMO + любая нейронная сеть

> Достоинства: отличный контекстуальный метод векторизации текстов, мощная модель

> Недостатки: сложность моделей

> Frameworks: `elmo`, `pytorch`

6) Bert + любая нейронная сеть

> Достоинства: отличный контекстуальный метод векторизации текстов, мощная модель

> Недостатки: сложность моделей

> Frameworks: `transformers`, `pytorch`

Вы также можете исследовать любые комбинации методов векторизации и моделей ML, которые сочтете нужными.

Ваша задача: провести сравнительный анализ не менее 3 алгоритмов классификации текстов. Сравнение стоит проводить по следующим параметрам:

- Качество классификации (актуальную метрику выберите самостоятельно)
- Время обучения модели
- Характерное время инференса модели

Данные можно загрузить по ссылке: https://drive.google.com/drive/folders/14hR7Pm2sH28rQttkD906PTLvtwHFLBRm?usp=sharing

Для упрощения Вашей работы предлагаем ряд функций для предобработки текстов.

In [ ]:
import re, string
regex = re.compile('[%s]' % re.escape(string.punctuation))
def clear(text: str) -> str:
    text = regex.sub('', text.lower())
    text = re.sub(r'[«»\n]', ' ', text)
    text = text.replace('ё', 'е')
    return text.strip()

In [ ]:
import nltk #natural language toolkit
from nltk.stem import WordNetLemmatizer

nltk.download('omw-1.4')
nltk.download('wordnet')

nltk.download('punkt') # без этой штуки мне выдает ошибку

lemmatizer = WordNetLemmatizer()

# проверяю, а то прикольно чет
print(lemmatizer.lemmatize("people"))
print(lemmatizer.lemmatize("worst"))
print(lemmatizer.lemmatize("confused"))
print(lemmatizer.lemmatize("interacting"))
print(lemmatizer.lemmatize("campers"))

people
worst
confused
interacting
camper


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
eng_stopwords = stopwords.words("english")

remove_stopwords = lambda tokenized_text, stopwords: [w for w in tokenized_text if not w in stopwords]
#type(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# $0)$ Предобработка данных

Для начала загрузим датасет и предобработаем его. Проделаем это здесь, а потом будем использовать во всех трех решениях задачи.

**Правда, я прописываю путь на свой гугл диск, так что с другого акканута, возможно, нужно прописать другой путь, чтобы файлы подгрузились. Не уверен.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/IMDB/"

In [ ]:
train_texts =[]
train_labels = []

test_texts =[]
test_labels = []


fp_train_texts = open(path+'train.texts','r',encoding='utf-8')
for text in fp_train_texts:
    train_texts.append(text)

fp_train_labels = open(path+'train.labels','r',encoding='utf-8')
for label in fp_train_labels:
    train_labels.append(label)

fp_test_texts = open(path+'dev.texts','r',encoding='utf-8')
for text in fp_test_texts:
    test_texts.append(text)

fp_test_labels = open(path+'dev.labels','r',encoding='utf-8')
for label in fp_test_labels:
    test_labels.append(label)


print('Длина тренировочного набора текстов: ', len(train_texts))
print('Длина тестового набора текстов: ',len(test_texts))

for i in range(5):
  print(train_labels[i])


Длина тренировочного набора текстов:  15000
Длина тестового набора текстов:  10000
neg

pos

neg

neg

pos



In [ ]:
# это просто проверка

#train_texts[7]
print(test_texts[5])

This movie surprised me in a good way. From the box I got the impression that it was an action thriller but it was too funny to be a thriller, even though it was somewhat exciting.<br /><br />There's a lot of nice one-liners and funny situations in this movie and James Belushi was born to do Bill Manucci, he does a great job. The rest of the cast ain't half-bad either and especially Timothy Dalton is a treat.<br /><br />The story can get pretty confusing at times as new characters shows up during the film. Things get more complicated as nobody seldom tells the truth about things. If you don't pay attention things might get a bit messy in the end but I really liked it.<br /><br />Louis Morneau isn't all that well known but he has done a perfectly OK job with this one and I never really grew impatient while watching the movie.<br /><br />Made men is well worth checking out.



In [ ]:
#предобработка текстов
preprocessed_texts=[]
preprocessed_test_texts=[]

for text in train_texts:
  text=clear(text)
  #prepr=text.split() Вроде бы так неправильно
  prepr=nltk.word_tokenize(text) # а так правильно
  prepr=remove_stopwords(prepr,eng_stopwords)
  for i in range(len(prepr)):
    prepr[i]=lemmatizer.lemmatize(prepr[i])
  preprocessed_texts.append(prepr)

for text in test_texts:
  text=clear(text)
  #prepr=text.split() Вроде бы так неправильно
  prepr=nltk.word_tokenize(text) # а так правильно
  prepr=remove_stopwords(prepr,eng_stopwords)
  for i in range(len(prepr)):
    prepr[i]=lemmatizer.lemmatize(prepr[i])
  preprocessed_test_texts.append(prepr)


In [ ]:
texts_train = [' '.join(t) for t in preprocessed_texts     ] # список уже полностью обработанных текстов
texts_test =  [' '.join(t) for t in preprocessed_test_texts]
print(preprocessed_texts[7],"\n","\n")

print(texts_test[5])

['say', 'worst', 'part', 'movie', 'first', 'half', 'hour', 'really', 'confused', 'example', 'bill', 'paxsons', 'character', 'long', 'hair', 'wearing', 'jacket', 'male', 'arrived', 'camp', 'turned', 'character', 'looked', 'like', 'bill', 'paxson', 'wasnt', 'said', 'wheres', 'bill', 'paxson', 'guy', 'girlfriend', 'said', '21', 'supposed', '20year', 'reunion', 'camp', 'director', 'alan', 'arkin', 'memorable', 'later', 'girl', 'interacting', 'talking', 'camp', 'experience', 'made', 'sense', 'would', 'one', 'year', 'old', 'said', 'movie', 'turned', 'pretty', 'good', 'kevin', 'pollak', 'nice', 'guy', 'always', 'teased', 'one', 'guy', 'complete', 'narcissist', 'ended', 'losing', 'beautiful', 'girlfriend', 'alan', 'arkin', 'interesting', 'oldstyle', 'camp', 'director', 'admits', 'grown', 'touch', 'modern', 'youth', 'best', 'part', 'none', 'grownup', 'camper', 'success', 'life', 'none', 'great', 'career', 'seemed', 'real', 'life', 'movie', 'compared', 'big', 'chill', 'way', 'wasnt', 'exciting',

Мы закончили с предобработкой данных.

# $1)$ Логистическая регрессия на основе векторов TF-IDF

Векторизуем тексты.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(texts_train) # Не понимаю, что именно делают эти 2 строчки
X_test=vectorizer.transform(texts_test) # Я просто хочу векторизовать тексты и инет + ноутбук Сережи пишут, что это нужная команда

print(type(X))
print(len(texts_train))
print(X.shape[0])
print(X[0])


<class 'scipy.sparse.csr.csr_matrix'>
15000
15000
  (0, 65795)	0.08252149888958198
  (0, 53973)	0.015200485498484724
  (0, 70969)	0.0731920817366284
  (0, 3928)	0.06272649433070067
  (0, 36209)	0.05901703148157166
  (0, 50475)	0.02201718762556578
  (0, 82274)	0.06877682986582323
  (0, 35783)	0.06063193987869852
  (0, 3557)	0.06582448954444176
  (0, 42468)	0.06649144221267984
  (0, 71627)	0.06304347287285315
  (0, 45575)	0.0588068714783407
  (0, 19200)	0.043893172458207515
  (0, 62896)	0.07190123651327343
  (0, 72903)	0.03805186965155708
  (0, 15951)	0.05030914649319414
  (0, 63477)	0.04781279531056748
  (0, 27710)	0.05116598958330504
  (0, 23959)	0.03309674157259758
  (0, 65858)	0.043804686902994106
  (0, 22354)	0.02315694993614921
  (0, 77834)	0.053441728432244984
  (0, 41677)	0.05179931794142777
  (0, 50986)	0.08922213841353052
  (0, 2529)	0.02903989369740796
  :	:
  (0, 65088)	0.04650529723311677
  (0, 28092)	0.07494045177747681
  (0, 10897)	0.040832886323145676
  (0, 31738)	0.05556

Обучаем логистическую регрессию, делаем предсказание и чекаем его качество на train.

In [ ]:
from sklearn.linear_model import LogisticRegression

logisticRegr = LogisticRegression()
logisticRegr.fit(X, train_labels)

predictions = logisticRegr.predict(X_test)
score = logisticRegr.score(X_test, test_labels)
print("Accuracy:",score)


Accuracy: 0.8825


# $2)$ Простой Байесовский классификатор

Теория: просто записываем теорему Байеса, откидываем знаменатель, переписываем вероятность текста как произведение вероятностей слов и набрасываем логарифм на обе части (чтобы бороться с чем-то очень близким к нулю). Считаем это и делаем вывод о принадлежности какому-то классу.

Но реализовывать уже сииииииииильно лень.